<a href="https://colab.research.google.com/github/pablosqvl/logtxt_sumar/blob/main/logtxt_sumar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Script para procesar errores de facturación
---
[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/pablosqvl/logtxt_sumar)

El código realiza lo siguiente:
1.   Carga archivos TXT y LOG desde una carpeta.
2.   Procesa los archivos TXT y extrae información de errores del archivo LOG.
3.   Limpia y une la información con descripciones de códigos desde un archivo del nomenclador.
4.   Asigna nombres de los efectores.
5.   Concatena los resultados en un DataFrame y lo exporta como un archivo excel con la fecha del día.

Antes de empezar:
1.   Descargar el [Nomenclador Vigente](https://sistemasmsp.misiones.gob.ar/nacer/modulos/reportes/nomencladores/nomencladores_vigentes.php) y renombrarlo "Nomenclador_Vigente.xls".
2.   Descargar los archivos txt y logs y guardarlos en una carpeta.



In [ ]:
# @title Cargar los archivos txt y logs {"display-mode":"form"}

import os
import pandas as pd
from datetime import datetime
from google.colab import files

# Definir carpetas de datos y salida
data_folder = "data/"
output_folder = "output/"

# Crear las carpetas si no existen
os.makedirs(data_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

#Subi archivos
uploaded = files.upload()
for fn in uploaded.keys():
  os.rename(fn, data_folder + fn)

In [ ]:
# @title Cargar el archivo con el nomenclador. Debe llamarse "Nomenclador_Vigente.xls"

# Cargar el nomenclador
uploaded = files.upload()
nomenclador = pd.read_excel("Nomenclador_Vigente.xls", skiprows=3)
nomenclador.columns = nomenclador.columns.str.lower().str.replace(' ', '_')
nomenclador = nomenclador.drop_duplicates(subset="codigo")[["codigo", "descripcion"]]

# Listar los archivos en la carpeta 'data' y filtrar los que no contengan '_LOG'
list_archivos = [f.replace('.TXT', '') for f in os.listdir(data_folder) if f.endswith('.TXT') and '_LOG' not in f]

# Nombres de las columnas
nombres_columnas = [
    "lit", "efector", "periodo", "data_0", "id_comprobante", "numero_beneficiario", "clase_dni", "tipo_dni", "num_dni",
    "apellido", "nombre", "fecha_nacimiento", "codigo_sumar", "fecha", "dato_vacuna1", "data_2", "data_3", "data_4",
    "data_5", "odonto", "data_7", "antitetanica_dosis1", "antitetanica_dosis2", "data_10", "data_11", "data_12",
    "data_13", "consultas_prenatales", "FPPECO", "FUM", "peso", "data_18", "talla", "data_20", "data_21", "data_22",
    "data_23", "data_24", "data_25", "data_26", "data_27", "data_28", "data_29", "data_30", "data_31", "data_32",
    "data_33", "data_34", "data_35", "data_36", "data_37", "data_38", "id_rismi", "data_40", "data_41", "profesional",
    "codigo_diagnostico", "diagnostico", "data_42", "tas", "tad", "data_45", "longitud_bebe", "perimetro_abdominal",
    "peso_clap", "sexo", "data_49", "data_50", "imc", "data_52", "data_53", "data_54", "domicilio", "altura",
    "barrio", "data_vacuna1", "ddata_vacuna2", "ddata_vacuna3", "data_vacuna4", "lote", "vencimiento_lote", "data_61",
    "data_62", "data_63", "data_64", "data_65", "data_66", "matricula", "tipo_dni_prof", "dni_prof", "nombre_profesional",
    "apellido_profesional"
]
# Leer cada archivo TXT y LOG, y almacenar los resultados
lista_dataframes = []

for file_name in list_archivos:
    # Cargar el archivo TXT
    txt_original = pd.read_csv(f"{data_folder}/{file_name}.TXT", delimiter=";", header=None, names=nombres_columnas, skiprows=1, dtype=str, encoding='latin-1') # Added encoding='latin-1' to handle different file encoding
    txt_original = txt_original.reset_index(drop=True).reset_index().rename(columns={"index": "id"})
    txt_original['id'] += 1

    # Cargar el archivo LOG
    with open(f"{data_folder}/{file_name}_LOG.TXT") as log_file:
        txt_log = log_file.readlines()

    errores = pd.DataFrame([line.strip() for line in txt_log if "linea Nro" in line], columns=['error'])
    errores['linea'] = errores['error'].str.extract(r'linea Nro (\d+)').astype(int)

    # Unir TXT con LOG
    txt_modificado = pd.merge(txt_original, errores, left_on='id', right_on='linea', how='right')
    txt_modificado['tipo_error'] = txt_modificado['error'].str.split('en la linea Nro').str[0]
    txt_modificado['descrip_error'] = txt_modificado['error'].str.extract(r'en la linea Nro \d+ (.+)')

    # Unir con nomenclador
    txt_modificado = txt_modificado.merge(nomenclador, left_on='codigo_sumar', right_on='codigo', how='left')

    # Asignar nombres a los efectores
    efectores = {
        "N00483": "SAN MARCOS", "N00296": "LOS PARAISOS", "N05448": "LATINOAMERICA", "N05449": "MIGUEL LANUS",
        "N05450": "DOS DE ABRIL", "N05455": "SAN GERARDO", "N95791": "BELEN", "N99980": "DON PEDRO",
        "N95809": "LAS ROSAS", "N99979": "COLONIA AEROPARQUE", "IME001": "IMeFiR", "N10450": "APS", "N00045": "LABORATORIO"
    }
    txt_modificado['nombre_efector'] = txt_modificado['efector'].map(efectores).fillna("")

    # Agregar a la lista de dataframes
    lista_dataframes.append(txt_modificado)

# Concatenar todos los dataframes
data_final = pd.concat(lista_dataframes, ignore_index=True)
data_final = data_final.dropna(axis=1, how='all')  # Eliminar columnas vacías
data_final = data_final[["nombre_efector"] + [col for col in data_final.columns if col != "nombre_efector"]]
# Ordenar columnas
column_to_move = data_final.pop("descrip_error")
data_final.insert(0, "descrip_error", column_to_move)
column_to_move = data_final.pop("tipo_error")
data_final.insert(0, "tipo_error", column_to_move)



Saving Nomenclador_Vigente.xls to Nomenclador_Vigente.xls


In [ ]:
# @title Descargar el archivo final a Excel

# Exportar a Excel
output_file_excel = os.path.join(output_folder, f"data_logs_{datetime.today().strftime('%d-%m-%y')}_result.xlsx")
data_final.to_excel(output_file_excel, index=False)

# Descargar el archivo output
files.download(output_file_excel)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>